<a href="https://colab.research.google.com/github/Laofin050/Tugas-Praktikum_049-050/blob/main/ProjectML_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Load Data**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/My Drive/ProjectPrakML"

In [ ]:
%cd /content/drive/My Drive/ProjectPrakML

/content/drive/My Drive/ProjectPrakML


In [ ]:
!ls

chest_xray  kaggle.json


In [ ]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

100% 2.29G/2.29G [00:45<00:00, 50.9MB/s]
100% 2.29G/2.29G [00:45<00:00, 53.7MB/s]


In [ ]:
!ls

chest-xray-pneumonia.zip  kaggle.json


In [ ]:
!unzip \*.zip &> /dev/null && rm *.zip

In [ ]:
!ls

chest_xray  kaggle.json


In [ ]:
from os import mkdir

mkdir ('New')

In [ ]:
!ls

chest_xray  kaggle.json  New


In [ ]:
import shutil,sys

shutil.move("/content/drive/My Drive/ProjectPrakML/chest_xray/chest_xray/test/NORMAL", "/content/drive/My Drive/ProjectPrakML/New")
shutil.move("/content/drive/My Drive/ProjectPrakML/chest_xray/chest_xray/test/PNEUMONIA", "/content/drive/My Drive/ProjectPrakML/New")

'/content/drive/My Drive/ProjectPrakML/New/PNEUMONIA'

**Preprocessing**

In [ ]:
%rm -rf test
%rm -rf train
%rm -rf valid

In [ ]:
import numpy as np
import pandas as pd
import os
from shutil import copyfile

TRAIN_DIR = "/content/drive/My Drive/ProjectPrakML/train/"
VALID_DIR ="/content/drive/My Drive/ProjectPrakML/valid/"
TEST_DIR ="/content/drive/My Drive/ProjectPrakML/test/"

os.mkdir(TRAIN_DIR)
os.mkdir(VALID_DIR)
os.mkdir(TEST_DIR)

for label in ['NORMAL', 'PNEUMONIA']:
    os.mkdir(TRAIN_DIR+label)
    os.mkdir(VALID_DIR+label)
    os.mkdir(TEST_DIR+label)

In [ ]:
import random

def train_valid_test_split(source, train_dir, valid_dir, test_dir,train_size, valid_size, test_size):
    # get files
    files = []
    for filename in os.listdir(source):
        file = source + filename
        if os.path.getsize(file) > 0:
            files.append(filename)
    
    # train valid split
    train_size = int(len(files) * (1-(valid_size + train_size)))
    valid_size = int(len(files) * (1-(valid_size + test_size)))
    test_size = int(len(files) * (1-train_size))
  
    # shuffle the dataset
    shuffled_files = random.sample(files, len(files))
    
    train_set = shuffled_files[train_size:valid_size]
    valid_set = shuffled_files[valid_size:]
    test_set = shuffled_files[:train_size]
    
    for filename in train_set:
        filepath = source + filename
        destination = train_dir + filename
        copyfile(filepath, destination)
        
    for filename in valid_set:
        filepath = source + filename
        destination = valid_dir + filename
        copyfile(filepath, destination)
    
    for filename in test_set:
        filepath = source + filename
        destination = test_dir + filename
        copyfile(filepath, destination)

SOURCE = "/content/drive/My Drive/ProjectPrakML/New/"

for label in ['NORMAL', 'PNEUMONIA']:
    print(label)
    train_valid_test_split(SOURCE+label+'/',  TRAIN_DIR +label+'/', VALID_DIR +label+'/', TEST_DIR +label+'/', train_size=0.8, valid_size=0.19, test_size=0.01)
    print('Total images: ', len(os.listdir(SOURCE +label+'/')))
    print('Training: ', len(os.listdir(TRAIN_DIR +label+'/')))
    print('Validation: ', len(os.listdir(VALID_DIR +label+'/')))
    print('Testing: ', len(os.listdir(TEST_DIR +label+'/')))
    print()

NORMAL
Total images:  234
Training:  185
Validation:  47
Testing:  2

PNEUMONIA
Total images:  390
Training:  309
Validation:  78
Testing:  3



**Augmentasi Data** 

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True, #bolak balik image secara vertika, dan true = on
    vertical_flip = True,
    rotation_range = 90,
    height_shift_range = 0.2,
    width_shift_range = 0.2,
    zoom_range = 0.2
)

test_generator = ImageDataGenerator(
    rescale = 1./255,
    validation_split = 0.7
)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_dir = '/content/drive/My Drive/ProjectPrakML/train'
train_datagen = ImageDataGenerator(
      rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (150, 150),
    color_mode='rgb',
    class_mode = 'categorical',
    shuffle=True,
)

Found 494 images belonging to 2 classes.


In [ ]:
from keras.preprocessing.image import ImageDataGenerator

valid_dir = '/content/drive/My Drive/ProjectPrakML/valid'
train_datagen = ImageDataGenerator(
      rescale=1./255)

valid_generator = train_datagen.flow_from_directory(
    valid_dir,
    target_size = (150, 150),
    color_mode='rgb',
    class_mode = 'categorical',
    shuffle=True,
)

Found 125 images belonging to 2 classes.


In [ ]:
from keras.preprocessing.image import ImageDataGenerator

test_dir = '/content/drive/My Drive/ProjectPrakML/test'
train_datagen = ImageDataGenerator(
      rescale=1./255)

test_generator = train_datagen.flow_from_directory(
    test_dir,
    target_size = (150, 150),
    color_mode='rgb',
    class_mode = 'categorical',
    shuffle=True,
)

Found 5 images belonging to 2 classes.


**Modelling**

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization
import tensorflow as tf

inputs = tf.keras.Input(shape = (150, 150, 3))
x = tf.keras.layers.Conv2D(filters = 16, kernel_size = (3, 3), activation = 'relu')(inputs)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu')(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu')(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu')(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.GlobalMaxPool2D()(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(512, activation = 'relu')(x)
outputs = tf.keras.layers.Dense(3, activation = 'softmax')(x)

model = tf.keras.Model(inputs = inputs, outputs = outputs)


model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)


history = model.fit(
    train_generator,
    validation_data = valid_generator,
    epochs = 100,
    callbacks = [
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=40,
            verbose=1, 
            mode='auto',
            cooldown=1
        )
    ]
)

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization
import tensorflow as tf

inputs = tf.keras.Input(shape = (150, 150, 3))
x = tf.keras.layers.Conv2D(filters = 16, kernel_size = (3, 3), activation = 'relu')(inputs)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu')(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu')(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu')(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.GlobalMaxPool2D()(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(512, activation = 'relu')(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
model2.add(Dense(3, activation='softmax'))
outputs = tf.keras.layers.Dense(3, activation = 'softmax')(x)

model = tf.keras.Model(inputs = inputs, outputs = outputs)


model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)


history = model.fit(
    train_generator,
    validation_data = valid_generator,
    epochs = 100,
    callbacks = [
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=40,
            verbose=1, 
            mode='auto',
            cooldown=1
        )
    ]
)